# QUESTIONS
1. RMSD for crossdocking: what whith what?

# Flexible Ligand - Flexible Protein Docking

# Theory: 

## Introduction
- Flexible ligand-flexible protein docking models the interaction between a ligand (e.g., drug candidate) and a protein receptor, allowing for conformational changes in both molecules.
- It is particularly useful for studying complex binding mechanisms, such as induced fit or allosteric modulation, where structural flexibility is key.

## Key Concepts
- **Molecular Flexibility**:
  - Proteins are not rigid; they undergo conformational changes to optimize interactions with ligands (induced fit hypothesis).
  - Ligands, especially small molecules, can explore numerous conformations due to bond rotations, allowing better complementarity to the protein binding site.
    
- **Degrees of Freedom**:
  - Flexible docking involves sampling multiple torsional angles for the ligand and accounting for side-chain and/or backbone movements in the protein.
  - While this adds complexity, it significantly improves docking accuracy.

## Simulation Workflow
1. **Ligand Preparation**:
   - Ligand flexibility is introduced by allowing bond rotations and conformational sampling.
   - Rotatable bonds and dihedral angles are defined for dynamic adjustments during docking.
2. **Protein Preparation**:
   - Define the side-chain flexibility: Commonly used for residues near the binding site.
4. **Search Algorithm**:
   - Algorithms such as Monte Carlo, Genetic Algorithms, or Molecular Dynamics are used to explore the conformational space of both the ligand and the protein.
   - Scoring functions evaluate binding affinity for each sampled conformation.
5. **Result Evaluation**:
   - Docking results typically consist of clusters of conformations ranked by predicted binding affinity and stability.
   - Analysis often includes identifying the most plausible binding mode and key molecular interactions (e.g., hydrogen bonds, hydrophobic contacts).


# Practice

1. **Implement flexible-protein / flexible-ligand docking with GNINA**

2. **Fetching of PDB files (see T015)**

3. **Preparation of protein using PDBFixer (develop on your own)**

4. **Selection of binding site (see T015)** 

5. **Selection of flexible residues (develop on your own, see GNINA documentation)** 

6. **Preparation of ligand (see T015)** 

7. **Docking into selected binding site (similar to T015 but check additional parameters)** 

8. **Calculation of RMSD relative to crystal ligand (develop on your own)** 

9. **Tools for comparing results** 

10. **RMSD between rigid and flexible ligand poses (develop on your own)** 

11. **Comparison of position/coordinates of protein residues (develop on your own)** 

In [1]:
### INSTALL IF NECCESSARY ###
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"
# !nvidia-smi
# !wget https://github.com/gnina/gnina/releases/download/v1.3/gnina
# !chmod +x gnina
# !./gnina --version
# # !pip install -q condacolab ### if running in colab ###
# # import condacolab
# # condacolab.install()
# # !conda --version
# !conda install conda-forge::pdbfixer
# !conda install conda-forge::nglview
# !conda install conda-forge::openbabel
# !conda install conda-forge::mdanalysis
# !conda install conda-forge::rdkit
# !conda install conda-forge::py3dmol



In [2]:
import os
import requests
import subprocess


import py3Dmol
from pdbfixer import PDBFixer
from openmm.app import PDBFile
from rdkit import Chem
from rdkit.Chem import AllChem, rdMolAlign
from Bio.PDB import PDBParser, Superimposer, PDBIO
from Bio.PDB.cealign import CEAligner
from openbabel import pybel
import subprocess

PDB_DIR = "pdb_files/"
LIGAND_DIR = "ligands/"
DOCKING_DIR = "docking_results/"
os.makedirs(PDB_DIR, exist_ok=True)
os.makedirs(LIGAND_DIR, exist_ok=True)
os.makedirs(DOCKING_DIR, exist_ok=True)



## Fetch PDB files

In [3]:
# --- 1. Fetch PDB Files ---

def fetch_pdb(pdb_id):
    url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
    response = requests.get(url)
    pdb_path = f"{PDB_DIR}{pdb_id}.pdb"
    
    if response.status_code == 200:
        with open(pdb_path, 'wb') as f:
            f.write(response.content)
        print(f"{pdb_id} downloaded.")
        return pdb_path
    else:
        raise Exception(f"Failed to fetch PDB {pdb_id}")
    
# Fetch and prepare proteins
pdb_4iaq = fetch_pdb("4IAQ")
pdb_6g79 = fetch_pdb("6G79")

4IAQ downloaded.
6G79 downloaded.


## Align Proteins

In [4]:
# --- 2. Align Protein Structures ---
def align_pdbs(reference_pdb, target_pdb):

    # Initialize PDB parser
    parser = PDBParser(QUIET=True)

    # Parse the structures
    ref_structure = parser.get_structure("ref", reference_pdb)
    target_structure = parser.get_structure("target", target_pdb)
    
    # Initialize CEAligner
    aligner = CEAligner()
    aligner.set_reference(ref_structure)
    
    # Perform CE alignment
    aligner.align(target_structure)

    # Save the aligned structures
    io = PDBIO()
    io.set_structure(ref_structure)
    io.save('pdb_files/4IAQ_aligned.pdb')

    io.set_structure(target_structure)
    io.save('pdb_files/6G79_aligned.pdb')
    
# Align proteins
align_pdbs(f"{PDB_DIR}4IAQ.pdb", f"{PDB_DIR}6G79.pdb")

In [5]:
# ## Check the alignment ###

# # Create 3Dmol.js viewer
# v = py3Dmol.view(width=1000, height=800)

# #Load protein structure
# v.addModel(open('pdb_files/4IAQ_aligned.pdb').read())  # Protein 1
# v.setStyle({'cartoon': {}, 'stick': {'radius': .1}})

# v.addModel(open('pdb_files/6G79_aligned.pdb').read())  # Protein 2
# v.setStyle({'cartoon': {}, 'stick': {'radius': .1}})

# v.rotate(90)  # Rotate for better visualization
# v.show()

### Extract true ligands (for RMSD calc later)

In [6]:
!grep 2GM pdb_files/4IAQ_aligned.pdb > ligands/true_2GM_lig.pdb

!grep EP5 pdb_files/6G79_aligned.pdb > ligands/true_EP5_lig.pdb

## Select Binding Sites

In [7]:
# --- 3. Select Binding Site ---

def select_binding_site(pdb_file, ligand_resname):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("protein", pdb_file)
    
    ligand_atoms = []
    for model in structure:
        for chain in model:
            for res in chain:
                if res.resname == ligand_resname:
                    for atom in res:
                        ligand_atoms.append(atom.coord)
    
    if not ligand_atoms:
        raise ValueError(f"Ligand {ligand_resname} not found in {pdb_file}")

    # Calculate center of mass for the ligand binding site
    center = sum(ligand_atoms) / len(ligand_atoms)
    print(f"Binding site center for {pdb_file}: {center}")
    return center

# Binding site selection
center_4iaq = select_binding_site(f"{PDB_DIR}4IAQ_aligned.pdb", "2GM")
center_6g79 = select_binding_site(f"{PDB_DIR}6G79_aligned.pdb", "EP5")

Binding site center for pdb_files/4IAQ_aligned.pdb: [-21.06556   10.188115  21.857306]
Binding site center for pdb_files/6G79_aligned.pdb: [-20.772429   9.942136  24.59037 ]


## Prepare the protein using PDBFixer

In [8]:
# --- 4. Prepare Protein with PDBFixer ---

def prepare_protein(input_pdb, output_pdb):
    fixer = PDBFixer(filename=input_pdb)
    fixer.findMissingResidues()
    fixer.findNonstandardResidues()
    fixer.replaceNonstandardResidues()
    fixer.removeHeterogens(True)
    fixer.findMissingAtoms()
    fixer.addMissingAtoms()
    fixer.addMissingHydrogens(7.0)
    with open(output_pdb, 'w') as f:
        PDBFile.writeFile(fixer.topology, fixer.positions, f)
    
    print(f"Protein {input_pdb} prepared as {output_pdb}")
    
prepare_protein(f"{PDB_DIR}4IAQ_aligned.pdb", f"{PDB_DIR}4IAQ_prepared.pdb")
prepare_protein(f"{PDB_DIR}6G79_aligned.pdb", f"{PDB_DIR}6G79_prepared.pdb")


Protein pdb_files/4IAQ_aligned.pdb prepared as pdb_files/4IAQ_prepared.pdb
Protein pdb_files/6G79_aligned.pdb prepared as pdb_files/6G79_prepared.pdb


## Prepare Ligands from SMILES

In [9]:
# --- 5. Prepare Ligand from SMILES (.sdf format) ---

def prepare_ligand_from_smiles(smiles, ligand_name):
    mol = Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)
    AllChem.EmbedMolecule(mol)
    ligand_path = f"{LIGAND_DIR}{ligand_name}.sdf"
    
    writer = Chem.SDWriter(ligand_path)
    writer.write(mol)
    writer.close()
    
    print(f"Ligand {ligand_name} saved as {ligand_path}")
    return ligand_path

ligand_2gm = prepare_ligand_from_smiles("CN1C[C@@H](C[C@H]2[C@H]1Cc3c[nH]c4cccc2c34)C(=O)N[C@]5(C)O[C@@]6(O)[C@@H]7CCCN7C(=O)[C@H](Cc8ccccc8)N6C5=O", "2GM")  # Example SMILES
ligand_ep5 = prepare_ligand_from_smiles("[NH3+]CCc1c[nH]c2ccc(OCC(=O)N3CCN(CC3)c4ccc(cc4)C#N)cc12", "EP5")

Ligand 2GM saved as ligands/2GM.sdf
Ligand EP5 saved as ligands/EP5.sdf


# Redocking
### We will utilize GNINA to perform redocking of ligands 2GM and EP5 with their proteins 4IAQ and 6G79 respectively.

## 4IAQ(rigid protein)+2GM(flexible ligand)

In [10]:
# --- Perform Docking with GNINA ---

def perform_docking(protein_pdb, ligand_sdf, center, output_sdf, size=(15, 15, 15), exhaustiveness=8, seed=123):
    """
    Perform docking using GNINA with a specified center and grid size.

    Args:
        protein_pdb (str): Path to the prepared protein PDB file.
        ligand_sdf (str): Path to the ligand SDF file.
        center (tuple): Coordinates for the docking box center (x, y, z).
        output_sdf (str): Path to the output SDF file.
        size (tuple): Dimensions of the docking grid (size_x, size_y, size_z).
        exhaustiveness (int): GNINA exhaustiveness parameter.
        seed (int): GNINA seed for reproducibility.
    """
    # Ensure output directories exist
    output_dir = os.path.dirname(output_sdf)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Build the GNINA command
    command = [
        "gnina",
        "-r", protein_pdb,
        "-l", ligand_sdf,
        "--center_x", str(center[0]),
        "--center_y", str(center[1]),
        "--center_z", str(center[2]),
        "--size_x", str(size[0]),
        "--size_y", str(size[1]),
        "--size_z", str(size[2]),
        "-o", output_sdf,
        "--exhaustiveness", str(exhaustiveness),
        "--seed", str(seed)
    ]
    
    # Run the command and capture the result
    result = subprocess.run(command, capture_output=True, text=True)
    
    # Check if the command was successful
    if result.returncode == 0:
        print(f"Docking complete: {output_sdf}")
    else:
        print(f"Error during docking: {result.stderr}")
    
    return output_sdf

# Example usage
protein_pdb = f"{PDB_DIR}4IAQ_prepared.pdb"
ligand_sdf = ligand_2gm
center = center_4iaq
output_sdf = "./docking_results/4IAQ_2GM_docked.sdf"

docked_4iaq_2gm = perform_docking(
    protein_pdb=protein_pdb,
    ligand_sdf=ligand_sdf,
    center=center,
    output_sdf=output_sdf
)


Docking complete: ./docking_results/4IAQ_2GM_docked.sdf


In [11]:
# --- Calculate RMSD ---

!obrms -f ligands/true_2GM_lig.pdb docking_results/4IAQ_2GM_docked.sdf

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is ligands/true_2GM_lig.pdb)

RMSD ligands/true_2GM_lig.pdb: 2.79803
RMSD ligands/true_2GM_lig.pdb: 0.968524
RMSD ligands/true_2GM_lig.pdb: 2.95261
RMSD ligands/true_2GM_lig.pdb: 4.29996
RMSD ligands/true_2GM_lig.pdb: 4.00169
RMSD ligands/true_2GM_lig.pdb: 4.01388
RMSD ligands/true_2GM_lig.pdb: 4.8997
RMSD ligands/true_2GM_lig.pdb: 4.38905
RMSD ligands/true_2GM_lig.pdb: 10.6386


In [12]:
### all modes
def show_all_modes(orig_prot, orig_lig, docking_lig):
    v = py3Dmol.view()
    v.addModel(open(orig_prot).read())
    v.setStyle({'cartoon':{},'stick':{'radius':.1}})
    v.addModel(open(orig_lig).read())
    v.setStyle({'model':1},{'stick':{'colorscheme':'dimgrayCarbon','radius':.125}})
    v.addModelsAsFrames(open(docking_lig).read())
    v.setStyle({'model':2},{'stick':{'colorscheme':'greenCarbon'}})
    v.zoomTo({'model':1})
    v.rotate(90)
    v.animate({'interval':2000})
    return v

show_all_modes(orig_prot='pdb_files/4IAQ_prepared.pdb', orig_lig='ligands/true_2GM_lig.pdb', docking_lig='docking_results/4IAQ_2GM_docked.sdf')

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

## 4IAQ(flexible protein)+2GM(flexible ligand)

In [13]:
# --- Perform Docking with GNINA ---

def perform_flex_docking(protein_pdb, ligand_sdf, center, output_sdf, flexdist, flexdist_ligand, out_flex, size=(15, 15, 15), exhaustiveness=8, seed=123):

    # Ensure output directories exist
    output_dir = os.path.dirname(output_sdf)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Build the GNINA command
    command = [
        "gnina",
        "-r", protein_pdb,
        "-l", ligand_sdf,
        "--center_x", str(center[0]),
        "--center_y", str(center[1]),
        "--center_z", str(center[2]),
        "--size_x", str(size[0]),
        "--size_y", str(size[1]),
        "--size_z", str(size[2]),
        "-o", output_sdf,
        "--flexdist", str(flexdist),
        "--flexdist_ligand", flexdist_ligand,
        "--out_flex", out_flex,
        "--exhaustiveness", str(exhaustiveness),
        "--seed", str(seed)
    ]
    
    # Run the command and capture the result
    result = subprocess.run(command, capture_output=True, text=True)
    
    # Check if the command was successful
    if result.returncode == 0:
        print(f"Docking complete: {output_sdf}")
    else:
        print(f"Error during docking: {result.stderr}")
    
    return output_sdf

# Example usage
protein_pdb = f"{PDB_DIR}4IAQ_prepared.pdb"
ligand_sdf = ligand_2gm
center = center_4iaq
output_sdf = "./docking_results/flex_4IAQ_2GM_docked.sdf"
flexdist = 4
flexdist_ligand = "ligands/true_2GM_lig.pdb"
out_flex = "flexout2_4IAQ.pdb"

docked_flex_4iaq_2gm = perform_flex_docking(
    protein_pdb=protein_pdb,
    ligand_sdf=ligand_sdf,
    center=center,
    output_sdf=output_sdf,
    flexdist=flexdist,
    flexdist_ligand=flexdist_ligand,
    out_flex=out_flex
)


Docking complete: ./docking_results/flex_4IAQ_2GM_docked.sdf


In [14]:
# --- Calculate RMSD ---

!obrms -f ligands/true_2GM_lig.pdb docking_results/flex_4IAQ_2GM_docked.sdf

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is ligands/true_2GM_lig.pdb)

RMSD ligands/true_2GM_lig.pdb: 2.79803
RMSD ligands/true_2GM_lig.pdb: 0.968524
RMSD ligands/true_2GM_lig.pdb: 2.95261
RMSD ligands/true_2GM_lig.pdb: 4.29996
RMSD ligands/true_2GM_lig.pdb: 4.00169
RMSD ligands/true_2GM_lig.pdb: 4.01388
RMSD ligands/true_2GM_lig.pdb: 4.8997
RMSD ligands/true_2GM_lig.pdb: 4.38905
RMSD ligands/true_2GM_lig.pdb: 10.6386


In [15]:
#show_all_modes(orig_prot='pdb_files/4IAQ_prepared.pdb', orig_lig='ligands/true_2GM_lig.pdb', docking_lig='docking_results/flex_4IAQ_2GM_docked.sdf')

## 6G79(rigid protein)+EP5(flexible ligand)

In [16]:
# --- Perform Docking with GNINA ---

protein_pdb = f"{PDB_DIR}6G79_prepared.pdb"
ligand_sdf = ligand_ep5
center = center_6g79
output_sdf = "./docking_results/6G79_EP5_docked.sdf"

docked_6g79_ep5 = perform_docking(
    protein_pdb=protein_pdb,
    ligand_sdf=ligand_sdf,
    center=center,
    output_sdf=output_sdf
)


Docking complete: ./docking_results/6G79_EP5_docked.sdf


In [17]:
# --- Calculate RMSD ---

!obrms -f ligands/true_EP5_lig.pdb docking_results/6G79_EP5_docked.sdf

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is ligands/true_EP5_lig.pdb)

RMSD ligands/true_EP5_lig.pdb: 1.38251
RMSD ligands/true_EP5_lig.pdb: 3.71583
RMSD ligands/true_EP5_lig.pdb: 2.42033
RMSD ligands/true_EP5_lig.pdb: 7.12812
RMSD ligands/true_EP5_lig.pdb: 2.79078
RMSD ligands/true_EP5_lig.pdb: 2.14469
RMSD ligands/true_EP5_lig.pdb: 8.9948
RMSD ligands/true_EP5_lig.pdb: 3.73585
RMSD ligands/true_EP5_lig.pdb: 7.18522


In [18]:
#show_all_modes(orig_prot="pdb_files/6G79_prepared.pdb", orig_lig='ligands/true_EP5_lig.pdb', docking_lig="docking_results/6G79_EP5_docked.sdf")

## 6G79(flexible protein)+EP5(flexible ligand)

In [19]:
# Example usage
protein_pdb = f"{PDB_DIR}6G79_prepared.pdb"
ligand_sdf = ligand_ep5
center = center_6g79
output_sdf = "./docking_results/flex_6G79_EP5_docked.sdf"
flexdist = 4
flexdist_ligand = "ligands/true_EP5_lig.pdb"
out_flex = "flexout4_6G79.pdb"



docked_flex_6g79_ep5= perform_flex_docking(
    protein_pdb=protein_pdb,
    ligand_sdf=ligand_sdf,
    center=center,
    output_sdf=output_sdf,
    flexdist=flexdist,
    flexdist_ligand=flexdist_ligand,
    out_flex=out_flex
)


Docking complete: ./docking_results/flex_6G79_EP5_docked.sdf


In [20]:
# --- Calculate RMSD ---

!obrms -f ligands/true_EP5_lig.pdb docking_results/flex_6G79_EP5_docked.sdf

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is ligands/true_EP5_lig.pdb)

RMSD ligands/true_EP5_lig.pdb: 1.38251
RMSD ligands/true_EP5_lig.pdb: 3.71583
RMSD ligands/true_EP5_lig.pdb: 2.42033
RMSD ligands/true_EP5_lig.pdb: 7.12812
RMSD ligands/true_EP5_lig.pdb: 2.79078
RMSD ligands/true_EP5_lig.pdb: 2.14469
RMSD ligands/true_EP5_lig.pdb: 8.9948
RMSD ligands/true_EP5_lig.pdb: 3.73585
RMSD ligands/true_EP5_lig.pdb: 7.18522


In [21]:
#show_all_modes(orig_prot="pdb_files/6G79_prepared.pdb", orig_lig='ligands/true_EP5_lig.pdb', docking_lig="docking_results/flex_6G79_EP5_docked.sdf")

### The results of re-docking experiments with rigid and flexible proteins are identical, as the binding pocket is already in its optimal conformation to accommodate the ligand effectively.

# Cross-Docking
### We will utilize GNINA to perform crossdocking of ligands EP5 and 2GM with proteins 4IAQ and 6G79 respectively.

## 4IAQ(rigid protein)+EP5(flexible ligand)

In [22]:
# --- Perform Docking with GNINA ---

protein_pdb = f"{PDB_DIR}4IAQ_prepared.pdb"
ligand_sdf = ligand_ep5
center = center_4iaq
output_sdf = "./docking_results/4IAQ_EP5_docked.sdf"

docked_4iaq_ep5 = perform_docking(
    protein_pdb=protein_pdb,
    ligand_sdf=ligand_sdf,
    center=center,
    output_sdf=output_sdf
)


Docking complete: ./docking_results/4IAQ_EP5_docked.sdf


In [23]:
# --- Calculate RMSD ---

!obrms -f ligands/true_EP5_lig.pdb docking_results/4IAQ_EP5_docked.sdf

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is ligands/true_EP5_lig.pdb)

RMSD ligands/true_EP5_lig.pdb: 2.65745
RMSD ligands/true_EP5_lig.pdb: 2.01983
RMSD ligands/true_EP5_lig.pdb: 4.23362
RMSD ligands/true_EP5_lig.pdb: 3.6398
RMSD ligands/true_EP5_lig.pdb: 6.67587
RMSD ligands/true_EP5_lig.pdb: 7.3185
RMSD ligands/true_EP5_lig.pdb: 8.92837
RMSD ligands/true_EP5_lig.pdb: 4.88379
RMSD ligands/true_EP5_lig.pdb: 3.67228


In [24]:
#show_all_modes(orig_prot="pdb_files/4IAQ_prepared.pdb", orig_lig='ligands/true_EP5_lig.pdb', docking_lig="docking_results/4IAQ_EP5_docked.sdf")

## 4IAQ(flexible protein)+EP5(flexible ligand)

In [25]:
# --- Perform Docking with GNINA ---

protein_pdb = f"{PDB_DIR}4IAQ_prepared.pdb"
ligand_sdf = ligand_ep5
center = center_4iaq
output_sdf = "./docking_results/flex_4IAQ_EP5_docked.sdf"
flexdist = 4
flexdist_ligand = "ligands/true_2GM_lig.pdb"
out_flex = "flexout6_4IAQ.pdb"



docked_flex_4iaq_ep5= perform_flex_docking(
    protein_pdb=protein_pdb,
    ligand_sdf=ligand_sdf,
    center=center,
    output_sdf=output_sdf,
    flexdist=flexdist,
    flexdist_ligand=flexdist_ligand,
    out_flex=out_flex
)

Docking complete: ./docking_results/flex_4IAQ_EP5_docked.sdf


In [26]:
# --- Calculate RMSD ---

!obrms -f ligands/true_EP5_lig.pdb docking_results/flex_4IAQ_EP5_docked.sdf

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is ligands/true_EP5_lig.pdb)

RMSD ligands/true_EP5_lig.pdb: 2.65745
RMSD ligands/true_EP5_lig.pdb: 2.01983
RMSD ligands/true_EP5_lig.pdb: 4.23362
RMSD ligands/true_EP5_lig.pdb: 3.6398
RMSD ligands/true_EP5_lig.pdb: 6.67587
RMSD ligands/true_EP5_lig.pdb: 7.3185
RMSD ligands/true_EP5_lig.pdb: 8.92837
RMSD ligands/true_EP5_lig.pdb: 4.88379
RMSD ligands/true_EP5_lig.pdb: 3.67228


In [27]:
#show_all_modes(orig_prot="pdb_files/4IAQ_prepared.pdb", orig_lig='ligands/true_EP5_lig.pdb', docking_lig="docking_results/flex_4IAQ_EP5_docked.sdf")

## 6G79(rigid protein)+2GM(flexible ligand)

In [28]:
# --- Perform Docking with GNINA ---

protein_pdb = f"{PDB_DIR}6G79_prepared.pdb"
ligand_sdf = ligand_2gm
center = center_6g79
output_sdf = "./docking_results/6G79_2GM_docked.sdf"

docked_6g79_2gn = perform_docking(
    protein_pdb=protein_pdb,
    ligand_sdf=ligand_sdf,
    center=center,
    output_sdf=output_sdf
)


Docking complete: ./docking_results/6G79_2GM_docked.sdf


In [29]:
# --- Calculate RMSD ---

!obrms -f ligands/true_2GM_lig.pdb docking_results/6G79_2GM_docked.sdf

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is ligands/true_2GM_lig.pdb)

RMSD ligands/true_2GM_lig.pdb: 8.30897
RMSD ligands/true_2GM_lig.pdb: 10.024
RMSD ligands/true_2GM_lig.pdb: 10.1473
RMSD ligands/true_2GM_lig.pdb: 10.5383
RMSD ligands/true_2GM_lig.pdb: 4.61248
RMSD ligands/true_2GM_lig.pdb: 10.9759
RMSD ligands/true_2GM_lig.pdb: 10.5625
RMSD ligands/true_2GM_lig.pdb: 10.4218
RMSD ligands/true_2GM_lig.pdb: 10.5671


In [30]:
#show_all_modes(orig_prot="pdb_files/6G79_prepared.pdb", orig_lig='ligands/true_2GM_lig.pdb', docking_lig="docking_results/6G79_2GM_docked.sdf")

## 6G79(flexible protein)+2GM(flexible ligand)

In [31]:
# --- Perform Docking with GNINA ---

protein_pdb = f"{PDB_DIR}6G79_prepared.pdb"
ligand_sdf = ligand_2gm
center = center_6g79
output_sdf = "./docking_results/flex_6G79_2GM_docked.sdf"
flexdist = 4
flexdist_ligand = "ligands/true_EP5_lig.pdb"
out_flex = "flexout8_6G79.pdb"



docked_flex_6g79_2gm= perform_flex_docking(
    protein_pdb=protein_pdb,
    ligand_sdf=ligand_sdf,
    center=center,
    output_sdf=output_sdf,
    flexdist=flexdist,
    flexdist_ligand=flexdist_ligand,
    out_flex=out_flex
)

Docking complete: ./docking_results/flex_6G79_2GM_docked.sdf


In [32]:
# --- Calculate RMSD ---

!obrms -f ligands/true_2GM_lig.pdb docking_results/flex_6G79_2GM_docked.sdf

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is ligands/true_2GM_lig.pdb)

RMSD ligands/true_2GM_lig.pdb: 8.30897
RMSD ligands/true_2GM_lig.pdb: 10.024
RMSD ligands/true_2GM_lig.pdb: 10.1473
RMSD ligands/true_2GM_lig.pdb: 10.5383
RMSD ligands/true_2GM_lig.pdb: 4.61248
RMSD ligands/true_2GM_lig.pdb: 10.9759
RMSD ligands/true_2GM_lig.pdb: 10.5625
RMSD ligands/true_2GM_lig.pdb: 10.4218
RMSD ligands/true_2GM_lig.pdb: 10.5671


In [34]:
show_all_modes(orig_prot="pdb_files/6G79_prepared.pdb", orig_lig='ligands/true_2GM_lig.pdb', docking_lig="docking_results/flex_6G79_2GM_docked.sdf")

3Dmol.js failed to load for some reason. Please check your browser console for error messages.